# Mini Project 1: Matching Pursuit
**Course**: CUHK MATH3320 - Foundation of Data Analytics
**Due Date**: October 15, 2024, 23:59:59
**Submission Platform**: Blackboard System

In this notebook, we will construct a regression model using the Matching Pursuit algorithm and the L0 norm constraint. We will go through the steps of finding a dataset, determining the optimal hyperparameter `k`, and implementing the Matching Pursuit algorithm.

## 0. Introduction
The goal of this project is to build a regression model using the Matching Pursuit algorithm while applying an L0 norm constraint to the model weights. The L0 norm constraint ensures that the number of non-zero coefficients in the model is controlled by a hyperparameter `k`. This helps in building a sparse model, reducing complexity, and potentially avoiding overfitting.

## 1. Dataset
For this project, we will use a publicly available dataset from the internet. In this example, we use the `California Housing` dataset available from the `sklearn` library. This dataset contains information on different factors that affect housing prices in California.

We will load the dataset and split it into training and testing sets.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np

# Load the dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print dataset shapes to verify
print(f'Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}')

Training set shape: (16512, 8), Testing set shape: (4128, 8)


The dataset has been successfully loaded and split into training and testing sets. The shapes of the training and testing sets are displayed above, ensuring that the data is ready for model training and evaluation.

## 2. Hyperparameter Selection
We need to determine the optimal value of the hyperparameter `k`, which controls the sparsity of our model. We will use cross-validation to select the best `k` value. The goal is to find a balance between model complexity and predictive accuracy by choosing an appropriate value for `k`.

We will use K-Fold cross-validation to evaluate different values of `k` and choose the one that yields the lowest error.

In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Define a function to perform cross-validation
def cross_validate_matching_pursuit(X, y, k_values, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    avg_errors = []

    for k in k_values:
        errors = []
        for train_index, val_index in kf.split(X):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            # Train the Matching Pursuit model
            w = matching_pursuit(X_train, y_train, k)
            y_pred = np.dot(X_val, w)
            error = mean_squared_error(y_val, y_pred)
            errors.append(error)

        avg_errors.append(np.mean(errors))

    return avg_errors

# Define a range of k values to test
k_values = range(1, X_train.shape[1] + 1)
# Placeholder for the matching_pursuit function (to be implemented later)
def matching_pursuit(X, y, k):
    pass  # Implementation of Matching Pursuit goes here

# Perform cross-validation (this will be revisited after implementing Matching Pursuit)
# avg_errors = cross_validate_matching_pursuit(X_train, y_train, k_values)

In the above cell, we have defined a function to perform cross-validation for different values of `k`. We used K-Fold cross-validation to ensure the model generalizes well. Once the Matching Pursuit algorithm is implemented, we will use this function to find the optimal value of `k`.

## 3. Matching Pursuit Algorithm Implementation
The Matching Pursuit algorithm is an iterative greedy algorithm that selects features one by one to approximate the target variable. Below, we provide the implementation of the algorithm.

In [8]:
def matching_pursuit(X, y, k):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    residual = y.copy()

    for _ in range(k):
        # Find the feature most correlated with the residual
        correlations = np.dot(X.T, residual)
        best_feature = np.argmax(np.abs(correlations))

        # Update the weight for the selected feature
        w[best_feature] += correlations[best_feature] / np.dot(X[:, best_feature], X[:, best_feature])

        # Update the residual
        residual -= w[best_feature] * X[:, best_feature]

    return w

# Now that we have implemented the algorithm, we can perform cross-validation to find the optimal k
avg_errors = cross_validate_matching_pursuit(X_train, y_train, k_values)
optimal_k = k_values[np.argmin(avg_errors)]
print(f'Optimal k: {optimal_k}')
print(f'Average errors for different k values: {avg_errors}')

Optimal k: 3
Average errors for different k values: [3.068931234392161, 2.393453602467726, 1.9798556390107827, 4.542450909295477, 8.544502435979698, 5.667090597583847, 2.0786370343565683, 2.3934536024677264]


We have implemented the Matching Pursuit algorithm and performed cross-validation to find the optimal `k`. The value of `k` that minimizes the cross-validation error is displayed above, along with the corresponding average errors.

## 4. Final Model and Evaluation
Using the optimal value of `k` obtained from cross-validation, we will train our Matching Pursuit model on the training dataset and evaluate it on the test dataset. The performance of the model will be assessed using the mean squared error (MSE) metric.

In [4]:
# Train the final model using the optimal k
final_w = matching_pursuit(X_train, y_train, optimal_k)

# Predict on the test set
y_pred = np.dot(X_test, final_w)

# Calculate the prediction error
test_error = mean_squared_error(y_test, y_pred)
print(f'Final prediction error on the test dataset: {test_error}')
print(f'Final model weights: {final_w}')

Final prediction error on the test dataset: 1.9120124681279305
Final model weights: [ 0.          0.          0.          0.          0.00052602  0.
  0.         -0.00687309]


The final model has been trained using the optimal value of `k`, and its performance on the test dataset has been evaluated. The mean squared error (MSE) and the model weights are displayed above. This allows us to analyze the predictive power of the model and understand which features were selected.

## 5. Conclusion
In this project, we implemented the Matching Pursuit algorithm to construct a regression model with an L0 norm constraint. We used cross-validation to determine the optimal value of the hyperparameter `k` and evaluated the final model on a test dataset.

The Matching Pursuit algorithm provides a way to build sparse regression models, which can be useful in scenarios where we want to limit model complexity or focus on the most important features. The model's performance was assessed using the mean squared error (MSE), and we observed the selected features through the final model weights. This approach can help in reducing overfitting and improving interpretability.

## 6. References
- Course textbook (Page 55) for the Matching Pursuit algorithm.
- Scikit-learn documentation: https://scikit-learn.org/